# take to .csv files (from eval and 5adv analyses) and merge them into one with all pairs

In [12]:
import numpy as np
from tqdm.auto import tqdm
from ap_rllib.config import get_trainer, get_config_by_name
import os
import pandas as pd
import ray

In [3]:
files = ['/tmp/to_eval_5adv.csv', '/tmp/to_eval_1adv_eval.csv']

In [38]:
dfs = [pd.read_csv(f) for f in files]

In [13]:
# need ray for parallel evaluation
ray.shutdown()
ray.init(num_cpus=28, ignore_reinit_error=True, log_to_driver=False)

2020-12-07 16:17:58,233	INFO services.py:1090 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '128.32.175.10',
 'raylet_ip_address': '128.32.175.10',
 'redis_address': '128.32.175.10:17811',
 'object_store_address': '/tmp/ray/session_2020-12-07_16-17-57_459964_4785/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-12-07_16-17-57_459964_4785/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2020-12-07_16-17-57_459964_4785',
 'metrics_export_port': 64283,
 'node_id': '4e53f1b392ad65b1998f15fa224ffefdb926d8aa'}

In [8]:
config = 'external_test'
config_ = get_config_by_name(config)
config_['_verbose'] = False
config_['_policies'] = [None, 'pretrained', 'pretrained']
trainer = get_trainer(config_)

Creating agent humanoid_blocker
Reading agent XML from: /scratch/sergei/better-adversarial-defenses/multiagent-competition/gym_compete/new_envs/assets/humanoid_body.xml
Creating agent humanoid
Reading agent XML from: /scratch/sergei/better-adversarial-defenses/multiagent-competition/gym_compete/new_envs/assets/humanoid_body.xml
Scene XML path: /scratch/sergei/better-adversarial-defenses/multiagent-competition/gym_compete/new_envs/assets/world_body.humanoid_body.humanoid_body.xml
Created Scene with agents
Creating agent humanoid_blocker
Reading agent XML from: /scratch/sergei/better-adversarial-defenses/multiagent-competition/gym_compete/new_envs/assets/humanoid_body.xml
Creating agent humanoid
Reading agent XML from: /scratch/sergei/better-adversarial-defenses/multiagent-competition/gym_compete/new_envs/assets/humanoid_body.xml
Scene XML path: /scratch/sergei/better-adversarial-defenses/multiagent-competition/gym_compete/new_envs/assets/world_body.humanoid_body.humanoid_body.xml
Create

2020-12-07 16:14:30,176	WARNING catalog.py:333 -- Custom ModelV2 should accept all custom options as **kwargs, instead of expecting them in config['custom_model_config']!


Setting NN weights
Loading normalization
Setting NN weights
Loading normalization
It looks like variables ["<tf.Varaible player_1/param/mean:0 shape=(380,) <dtype: 'float32'>>", "<tf.Varaible dense/bias:0 shape=(64,) <dtype: 'float32'>>", "<tf.Varaible player_1/sequential/h1/kernel:0 shape=(380, 64) <dtype: 'float32'>>", "<tf.Varaible player_1/param/clip:0 shape=(1,) <dtype: 'float32'>>", "<tf.Varaible player_1/sequential/value/bias:0 shape=(1,) <dtype: 'float32'>>", "<tf.Varaible mean/bias:0 shape=(17,) <dtype: 'float32'>>", "<tf.Varaible player_1/var/std:0 shape=(1, 17) <dtype: 'float32'>>", "<tf.Varaible player_1/param/mean:0 shape=(380,) <dtype: 'float32'>>", "<tf.Varaible dense/kernel:0 shape=(380, 64) <dtype: 'float32'>>", "<tf.Varaible dense_1/kernel:0 shape=(64, 64) <dtype: 'float32'>>", "<tf.Varaible player_1/sequential/h2/kernel:0 shape=(64, 64) <dtype: 'float32'>>", "<tf.Varaible player_1/sequential/h2/bias:0 shape=(64,) <dtype: 'float32'>>", "<tf.Varaible player_1/param/std

2020-12-07 16:14:30,739	WARNING catalog.py:333 -- Custom ModelV2 should accept all custom options as **kwargs, instead of expecting them in config['custom_model_config']!


Setting NN weights
Loading normalization
Setting NN weights
Loading normalization
It looks like variables ["<tf.Varaible dense_4/kernel:0 shape=(380, 64) <dtype: 'float32'>>", "<tf.Varaible dense_5/bias:0 shape=(64,) <dtype: 'float32'>>", "<tf.Varaible player_2/sequential_2/value/bias:0 shape=(1,) <dtype: 'float32'>>", "<tf.Varaible player_2/param/std:0 shape=(380,) <dtype: 'float32'>>", "<tf.Varaible player_2/param/std:0 shape=(1,) <dtype: 'float32'>>", "<tf.Varaible player_2/sequential_2/h1/kernel:0 shape=(380, 64) <dtype: 'float32'>>", "<tf.Varaible player_2/param/std:0 shape=(380,) <dtype: 'float32'>>", "<tf.Varaible player_2/sequential_2/h1/bias:0 shape=(64,) <dtype: 'float32'>>", "<tf.Varaible player_2/param/clip:0 shape=(1,) <dtype: 'float32'>>", "<tf.Varaible mean_2/kernel:0 shape=(64, 17) <dtype: 'float32'>>", "<tf.Varaible mean_2/bias:0 shape=(17,) <dtype: 'float32'>>", "<tf.Varaible player_2/sequential_2/h2/kernel:0 shape=(64, 64) <dtype: 'float32'>>", "<tf.Varaible player_2

Install gputil for GPU system monitoring.


Creating agent humanoid_blocker
Reading agent XML from: /scratch/sergei/better-adversarial-defenses/multiagent-competition/gym_compete/new_envs/assets/humanoid_body.xml
Creating agent humanoid
Reading agent XML from: /scratch/sergei/better-adversarial-defenses/multiagent-competition/gym_compete/new_envs/assets/humanoid_body.xml
Scene XML path: /scratch/sergei/better-adversarial-defenses/multiagent-competition/gym_compete/new_envs/assets/world_body.humanoid_body.humanoid_body.xml
Created Scene with agents


In [23]:
import logging
logging.basicConfig(level=logging.WARNING)

In [ ]:
policies = ['player_1', 'player_2']
data = {p: [] for p in policies} # format: p -> [(name, hash, weights)]

for f in tqdm(files):
    for _, r in tqdm(pd.read_csv(f).iterrows(), leave=False):
        trainer.restore(r.checkpoint_rllib)
        for p in policies:
            name = r[p]
            weights = trainer.get_weights()[p]
            hash_ = sum([np.sum(t) for t in weights])
            data[p].append((name, hash_, weights))

In [27]:
unique_names = {p: set(d[0] for d in data[p]) for p in policies}

In [33]:
# checking that hashes match for the same name
for p in policies:
    for n in unique_names[p]:
        vals = [d for d in data[p] if d[0] == n]
        hashes = [v[1] for v in vals]
        assert all([hashes[0] == h for h in hashes])

In [34]:
# only unique names
data_u = {p: [] for p in policies}
for p in policies:
    for n in unique_names[p]:
        val = [d for d in data[p] if d[0] == n][0]
        data_u[p].append(val)

In [39]:
# checking that got all the agents
total_players_df = {p: sum([len(set(df[p])) for df in dfs]) for p in policies}
for p in policies:
    assert len(data_u[p]) == total_players_df[p]

In [45]:
from uuid import uuid1

In [46]:
# saving the output!
ds = []
for p1name, _, p1w in data_u['player_1']:
    for p2name, _, p2w in data_u['player_2']:
        d = {'player_1': p1name, 'player_2': p2name}
        trainer.set_weights({'player_1': p1w, 'player_2': p2w})
        ckpt = trainer.save(f"/tmp/{uuid1()}")
        w_hash = sum([sum([np.sum(t) for t in y]) for y in trainer.get_weights().values()])
        d['checkpoint_rllib'] = ckpt
        d['hash'] = w_hash
#         print(p1name, p2name)
    
        for d_other in ds:
            if d_other['hash'] == d['hash']:
                print("Potential collision", d, d_other)
                
        ds.append(d)

In [47]:
hashes = [x['hash'] for x in ds]
assert len(hashes) == len(set(hashes)), "Some pairs are the same"

In [48]:
fn = "/tmp/to_eval_all.csv"
df_to_eval = pd.DataFrame(ds)
df_to_eval.to_csv(fn, index=False)

In [49]:
df_to_eval

,player_1,player_2,checkpoint_rllib,hash
0,0,5504,/tmp/a6d0e9d2-38ec-11eb-85ce-1831bf455a80/chec...,4558.288290
1,0,5505,/tmp/a6d0e9d3-38ec-11eb-85ce-1831bf455a80/chec...,4642.882576
2,0,5506,/tmp/a6d0e9d4-38ec-11eb-85ce-1831bf455a80/chec...,4681.178183
3,0,5507,/tmp/a6d0e9d5-38ec-11eb-85ce-1831bf455a80/chec...,4660.265835
4,0,5508,/tmp/a6d0e9d6-38ec-11eb-85ce-1831bf455a80/chec...,4688.704487
...,...,...,...,...
4055,42,5111,/tmp/b5e5c97d-38ec-11eb-85ce-1831bf455a80/chec...,4605.629721
4056,42,5112,/tmp/b5e5c97e-38ec-11eb-85ce-1831bf455a80/chec...,4634.975891
4057,42,5500,/tmp/b5e5c97f-38ec-11eb-85ce-1831bf455a80/chec...,4625.577477
4058,42,5502,/tmp/b5e5c980-38ec-11eb-85ce-1831bf455a80/chec...,4615.390228
